# Real-World Applications: TF-IDF
In this task Hadoop Streaming is used to process Wikipedia articles dump (/data/wiki/en_articles_part).

The purpose of this task is to calculate <code>tf*idf</code> for each pair (word, article) from the Wikipedia dump. Apply the stop words filter to speed up calculations. Term frequency (tf) is a function depending on a term (word) and a document (article):

<code>tf(term, doc_id) = Nt/N</code>,

where Nt - quantity of particular term in the document, N - the total number of terms in the document (without stop words)

Inverse document frequency (idf) is a function depends on a term:

<code>idf(term) = 1/log(1 + Dt)</code>,

where Dt - number of documents in the dataset with the particular term.

You can find more information here: https://en.wikipedia.xn--org/wiki/Tfidf-q82h but use just the formulas mentioned above.

Dataset location: /data/wiki/en_articles_part

Stop words list is in ‘/datasets/stop_words_en.txt’ file.

Format: article_id <i>tab</i> article_text

To parse the articles don’t forget about Unicode (even though this is an English Wikipedia dump, there are many characters from other languages), remove punctuation marks and transform words to lowercase to get the correct quantities. To cope with Unicode we recommend to use the following tokenizer:

Output: <code>tf*idf</code> for term=’labor’ and article_id=12

The result on the sample dataset:

<code>0.000351</code>

In [79]:
%%writefile test.dat

1	In this task Hadoop Streaming is used to process Wikipedia articles dump calculations.
2	The purpose of this task is to calculate tf*idf for each pair (word, article) from the Wikipedia dump calculations.
3	Apply the stop words filter to speed up calculations calculations calculations calculations calculations. 
4	Term frequency (tf) is a function depending on a term (word) and a document (article)
5	To parse the articles don’t forget about Unicode (even though this is an English Wikipedia dump, there are many characters from other languages), remove punctuation marks and transform words to lowercase to get the correct quantities.

Overwriting test.dat


In [82]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

with open('/datasets/stop_words_en.txt', "r") as f:
    stop_words = set(f.read().splitlines())
    
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    text= re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    text = re.split("\W*\s+\W*", text, flags=re.UNICODE)   
    
    tf = dict()
    for w in text:
        w = w.lower()
        if w in stop_words:
            continue
            
        if w in tf:
            tf[w] += 1
        else:
            tf[w] = 1
            
    total = len(tf)    
    for term, count in tf.iteritems():
        print "%s\t%s\t%f" % (term, article_id, float(count)/total) 

Overwriting mapper1.py


In [83]:
cat test.dat | python2 ./mapper1.py | head

articles	1	0.111111
task	1	0.111111
dump	1	0.111111
process	1	0.111111
wikipedia	1	0.111111
hadoop	1	0.111111
streaming	1	0.111111
used	1	0.111111
calculations	1	0.111111
tf*idf	2	0.100000


In [85]:
%%writefile reducer1.py

import sys
from math import log

current_key = None
cache = dict() # TODO try to use join to avoid in-memory collection

def process_record(term, cache):
    idf = 1/log(1+len(cache))
    for article_id, tf in cache.iteritems():
        print "%s\t%s\t%f" % (term, article_id, tf*idf)

for line in sys.stdin:
    try:
        term, article_id, tf = line.strip().split('\t', 2)
        tf = float(tf)
    except ValueError as e:
        continue
        
    if current_key != term:
        if current_key:            
            process_record(current_key, cache)
        current_key = term
        cache.clear()
    cache[article_id] = tf # TODO use list
    
if current_key:
    process_record(current_key, cache)

Overwriting reducer1.py


In [86]:
cat test.dat | python2 ./mapper1.py | sort | python2 ./reducer1.py | tail

transform	5	0.080150
unicode	5	0.080150
used	1	0.160299
wikipedia	1	0.080150
wikipedia	2	0.072135
wikipedia	5	0.040075
word	2	0.091024
word	4	0.113780
words	3	0.151707
words	5	0.050569
